In [21]:
library('tidyverse', warn.conflicts = FALSE)
library('robustbase')


In [15]:
master <- "/Users/laurituominen/Documents/Research/Reettis/new_analyses/master/"
dat <- read_csv(paste0(master, 'merged_data.csv'),show_col_types = FALSE)

dat <- dat %>% mutate(Sex = factor(Sex, levels = c('M', 'F')),
                      group =factor(Group, levels=c('CTR', 'CHR', 'FEP')))

# exposure vs cortical thickness 
dat %>% filter(group %in% c('FEP', 'CHR')) %>% 
  lmrob(mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group, na.action=na.omit, data=.) %>%
  summary()


Call:
lmrob(formula = mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group, 
    data = ., na.action = na.omit)
 \--> method = "MM"
Residuals:
      Min        1Q    Median        3Q       Max 
-0.193418 -0.060363 -0.002415  0.060400  0.197560 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           2.636e+00  3.093e-02  85.234  < 2e-16 ***
Lifetime_ap_exposure -1.127e-06  2.719e-07  -4.146 6.17e-05 ***
Age                  -2.814e-03  1.070e-03  -2.631  0.00959 ** 
SexF                 -3.190e-02  1.443e-02  -2.211  0.02887 *  
GroupFEP             -1.975e-02  1.526e-02  -1.294  0.19795    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Robust residual standard error: 0.08622 
Multiple R-squared:  0.1828,	Adjusted R-squared:  0.1569 
Convergence in 9 IRWLS iterations

Robustness weights: 
 15 weights are ~= 1. The remaining 116 ones are summarized as
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5788  0.8933

In [36]:
# covariates in sensitivity analyses 
covars <- c("Total_symptoms_score", "Positive_symptoms_score", "Negative_symptoms_score", "BMI", "Hospital_days", "Times_admitted",
            "GAF", "SOFAS")

# estimate the effects including one of the sensitivity covariates in the model at the time 
vals <- matrix(nrow=length(covars), ncol=4 )
for (i in seq(from=1,to=length(covars))){
  formula <- reformulate(termlabels =  c("Lifetime_ap_exposure","Age","Sex","Group", covars[i]), response = 'mean_thickness')
  m <- dat %>% filter(group %in% c('FEP', 'CHR')) %>%
    lmrob(formula = formula, data=.) %>%
    summary()
  vals[i,] <- as.vector(m$coefficients[2,]) # get parameters for "Lifetime_ap_exposure"
}

# make printable 
df <- as.data.frame(vals)
df$covariate <- covars 
df <- df[,c(5,1,2,3,4)]
names(df) <- c("Covariate", "Estimate", "Std. Error", "t value", "Pr(>|t|)")
# print out the parameters for "Lifetime_ap_exposure" of each of the sensitivity model 
df %>% format(scientific=FALSE)

,Covariate,Estimate,Std. Error,t value,Pr(>|t|)
,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>
1,Total_symptoms_score,-0.000001196205,0.0000002510025,-4.765708,0.000005329216
2,Positive_symptoms_score,-0.000001200869,0.0000002611666,-4.598097,0.000010636766
3,Negative_symptoms_score,-0.000001206891,0.0000002486682,-4.853419,0.000003689690
4,BMI,-0.000001088181,0.0000002533377,-4.295375,0.000034697531
5,Hospital_days,-0.000001040095,0.0000003143459,-3.308759,0.001226917059
6,Times_admitted,-0.000001050679,0.0000002826816,-3.716827,0.000303968242
7,GAF,-0.000001163543,0.0000002824823,-4.118996,0.000068693240
8,SOFAS,-0.000001163934,0.0000002786524,-4.177011,0.000054987136
